**CONNECTION**

**This code cell Establishes a Connection to the PostgreSQL Database and Executing query for tables and data model in the database**
It requires necessary connection parameters such as the host, database name, username, password, and port number.

In [ ]:
import psycopg2

# Function to connect to the PostgreSQL database
def connect_to_database():
    try:
        conn = psycopg2.connect(
                host='localhost',
                database='food_store',
                user='postgres',
                password='root',
        )
        print("Database connection successful!")
        return conn
    except psycopg2.Error as e:
        print("Error: Unable to connect to the database.")
        print(e)
        return None

# Function to execute a query and fetch all rows
def execute_query_fetch_all(query):
    conn = connect_to_database()
    if conn:
        try:
            cur = conn.cursor()
            cur.execute(query)
            rows = cur.fetchall()
            conn.close()
            return rows
        except psycopg2.Error as e:
            print(e)
            return None
    else:
        return None

# Function to execute a query without fetching results
def execute_query(query):
    conn = connect_to_database()
    if conn:
        try:
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
            conn.close()
            print("Query executed successfully!")
        except psycopg2.Error as e:
            print(e)

            # Creating the tables in the database
create_tables_query = """
    CREATE TABLE IF NOT EXISTS shoppers (
        shopper_id SERIAL PRIMARY KEY,
        shopper_name VARCHAR(255) NOT NULL,
        payment_method VARCHAR(20) NOT NULL
    );

    CREATE TABLE IF NOT EXISTS shoppingtrips (
        trip_id SERIAL PRIMARY KEY,
        shopper_id INTEGER REFERENCES shoppers(shopper_id),
        trip_date DATE NOT NULL,
        is_identifiable BOOLEAN NOT NULL,
        pre_order BOOLEAN NOT NULL
    );

    CREATE TABLE IF NOT EXISTS departments (
        department_id SERIAL PRIMARY KEY,
        department_name VARCHAR(100) NOT NULL
    );

    CREATE TABLE IF NOT EXISTS items (
        item_id SERIAL PRIMARY KEY,
        item_name VARCHAR(255) NOT NULL,
        department_id INTEGER REFERENCES departments(department_id)
    );

    CREATE TABLE IF NOT EXISTS purchases (
        purchase_id SERIAL PRIMARY KEY,
        trip_id INTEGER REFERENCES shoppingtrips(trip_id),
        item_id INTEGER REFERENCES items(item_id),
        amount INTEGER NOT NULL
    );
"""

# Insert test data into the tables
insert_data_query = """
    -- Departments
    INSERT INTO departments (department_name) VALUES
        ('household'),
        ('personal goods'),
        ('dried goods'),
        ('fresh vegetables'),
        ('supplements');

    -- Items
    INSERT INTO items (item_name, department_id) VALUES
        ('item1', 1),
        ('item2', 2),
        ('item3', 3),
        ('item4', 4),
        ('item5', 5);

    -- Shoppers
    INSERT INTO shoppers (shopper_name, payment_method) VALUES
        ('Chris Brown', 'card'),
        ('Jonhson Trevor', 'cash'),
        ('Anonymous Shopper', 'cash');

    -- Shopping Trips
    INSERT INTO shoppingtrips (shopper_id, trip_date, is_identifiable, pre_order) VALUES
        (1, '2023-07-19', TRUE, TRUE),
        (2, '2023-07-19', TRUE, FALSE),
        (NULL, '2023-07-19', FALSE, FALSE);

   -- Purchases
INSERT INTO purchases (trip_id, item_id, amount) VALUES
    (1, 3, 2),   -- Shopper 1, Trip 1: Purchased dried goods (item3)
    (1, 4, 1),   -- Shopper 1, Trip 1: Purchased fresh vegetables (item4)
    (2, 3, 1),   -- Shopper 2, Trip 1: Purchased dried goods (item3)
    (2, 4, 2),   -- Shopper 2, Trip 1: Purchased fresh vegetables (item4)
    (3, 3, 1),   -- Shopper 3, Trip 1: Purchased dried goods (item3)
    (3, 5, 1);   -- Shopper 3, Trip 1: Purchased supplements (item5)

"""

# Execution of the table creation and data insertion queries
execute_query(create_tables_query)
execute_query(insert_data_query)

**ONE**

**Listing Tables in the Database**

This code cell defines a function to list all the tables in the database. It executes a SQL query to fetch the table names from the database and displays them as the output.

In [ ]:
query_list_tables = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
"""

tables = execute_query_fetch_all(query_list_tables)
print("Tables in the database:")
print(tables)


**TWO**

**Listing Items in the Supplements Section**
This code cell defines a function to list all the items in the supplements section. 

In [ ]:
query_list_supplements = """
    SELECT item_name
    FROM items
    WHERE department_id = (SELECT department_id FROM departments WHERE department_name = 'supplements');
"""

supplements = execute_query_fetch_all(query_list_supplements)
print("\nItems in the supplements section:")
print(supplements)


**THREE**

**Listing All Departments**
This code cell defines a function to list all the departments in the health food store. 

In [ ]:
query_list_departments = """
    SELECT department_name
    FROM departments;
"""

departments = execute_query_fetch_all(query_list_departments)
print("\nDepartments:")
print(departments)


**FOUR**

**Adding an Anonymous Shopping Trip**
This code cell defines a function to add an anonymous shopping trip to the database.

In [ ]:
query_add_anonymous_trip = """
    INSERT INTO shoppingtrips (shopper_id, trip_date, is_identifiable, pre_order)
    VALUES (NULL, '2023-07-19', FALSE, FALSE);

    INSERT INTO purchases (trip_id, item_id, amount)
    VALUES (currval('shoppingtrips_trip_id_seq'), (SELECT item_id FROM items WHERE item_name = 'item3'), 1);
"""

execute_query(query_add_anonymous_trip)


 **FIVE**

 **Selecting Shopping Trips with Pre-Orders**
This code cell defines a function to select all the shopping trips that have purchased pre-orders. 

In [ ]:
query_select_pre_orders = """
    SELECT st.trip_id, s.shopper_id, s.shopper_name, st.trip_date
    FROM shoppingtrips st
    JOIN shoppers s ON st.shopper_id = s.shopper_id
    WHERE st.pre_order = TRUE;
"""

pre_orders = execute_query_fetch_all(query_select_pre_orders)
print("\nShopping trips with pre-orders:")
print(pre_orders)


**SIX**

**This function cell finds identifiable shoppers who purchased both dried goods and fresh vegetables in the same trip**
 Executing the query to find identifiable shoppers who purchased both dried goods and fresh vegetables

In [ ]:
query_find_shoppers_dried_goods_and_vegetables = """
    SELECT s.shopper_id, s.shopper_name
    FROM shoppers s
    JOIN shoppingtrips st ON s.shopper_id = st.shopper_id
    JOIN purchases p ON st.trip_id = p.trip_id
    JOIN items i ON p.item_id = i.item_id
    WHERE i.department_id IN (
        SELECT department_id FROM departments WHERE department_name IN ('dried goods', 'fresh vegetables')
    )
    GROUP BY s.shopper_id, s.shopper_name
    HAVING COUNT(DISTINCT i.department_id) = 2;
"""

# Executing the query to find identifiable shoppers who purchased both dried goods and fresh vegetables

shoppers_dried_goods_vegetables = execute_query_fetch_all(query_find_shoppers_dried_goods_and_vegetables)
print("\nIdentifiable shoppers who purchased both dried goods and fresh vegetables in the same trip:")
print(shoppers_dried_goods_vegetables)


**SEVEN**


**This cell Function deletes the anonymous trip and associated purchases, from the shoppingtrips table**
 Calling the function to delete the anonymous trip and associated purchases

In [ ]:
def delete_anonymous_trip():
    # delete the associated purchases for the anonymous trip
    query_delete_purchases = """
        DELETE FROM purchases WHERE trip_id IN (
            SELECT trip_id FROM shoppingtrips WHERE shopper_id IS NULL AND trip_date = '2023-07-19'
        );
    """
    execute_query(query_delete_purchases)

    #  delete the anonymous trip from the shoppingtrips table
    query_delete_trip = """
        DELETE FROM shoppingtrips WHERE shopper_id IS NULL AND trip_date = '2023-07-19';
    """
    execute_query(query_delete_trip)
    print("Anonymous trip DELETED SUCCESSFULLY!")

# Calling the function to delete the anonymous trip and associated purchases
delete_anonymous_trip()
